### Colliding two objects together within one SSP

Demonstrates sending two objects towards each other in a single SSP. Each step we clean up the position of both objects, update them separately, rebind with their object pointers, and then add back together. This is described in more detail by the following math. 

SSP representation:

$\text{SSP} = \text{OBJ}_1 \ast X^{x_1} \ast Y^{y_1} + \text{OBJ}_2 \ast X^{x_2} \ast Y^{y_2}$

Continuous cleanup function (single layer of 512 ReLUs):

$f(X^x \ast Y^y + \eta) \approx X^x \ast Y^y$

Each iteration:

$\text{SSP} \leftarrow \text{OBJ}_1 \ast f(\text{SSP} \ast \text{OBJ}_1^{-1}) \ast X^{\Delta x_1} \ast Y^{\Delta y_1} + \text{OBJ}_2 \ast f(\text{SSP} \ast \text{OBJ}_2^{-1}) \ast X^{\Delta x_2} \ast Y^{\Delta y_2}$

To make this work well (e.g., across 500 iterations), the cleanup data used for training has the ideal scaling (default $\sigma = 0.1$, which is normalized by $1/\sqrt{d}$) on the normal noise, and the cleanup always return a unitary vector.

The final plot compares the above approach (left) to the ground truth where objects are updated separately (middle), and shows the difference between the two (right).

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

from ssp.cleanup import Cleanup, generate_cleanup_data
from ssp.maps import Spatial2D
from ssp.plots import heatmap_animation, create_gif

In [ ]:
dim = 768
names = ["A", "B"]
map_radius = 5  # half-length of square centered about (0, 0)

In [ ]:
ssp_map = Spatial2D(dim=dim, scale=1, rng=np.random.RandomState(seed=0))
ssp_map.build_grid(x_len=map_radius, y_len=map_radius,
                   x_spaces=101, y_spaces=101, centered=True)

In [ ]:
cleanup = Cleanup(model=dim, vocab=ssp_map.voc)
cleanup.train(objs=names, low=-map_radius, high=map_radius)

In [ ]:
valid_x, valid_y = generate_cleanup_data(
    ssp_map.voc, names, -map_radius, map_radius)

plt.figure()
plt.title("Cleanup /w Single Layer ReLU")
plt.plot(cleanup.model.costs, label="Training")
plt.hlines([cleanup.model.cost(cleanup.model(valid_x), valid_y)],
           0, len(cleanup.model.costs) - 1,
           label="Validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Cost")
plt.yscale('log')
plt.show()

In [ ]:
dt = 0.002
objp = [(-4, -3), (0, 5)]  # initial positions
objv = [(4, 3), (0, -5)]  # initial velocity

ground_ssp = []  # ground truth for each object
T = []  # translational shift for each object
for i in range(len(names)):
    ground_ssp.append(
        ssp_map.encode_point(objp[i][0], objp[i][1], name=names[i])
    )
    T.append(ssp_map.encode_point(dt*objv[i][0], dt*objv[i][1], name=None))

ssp = np.sum(ground_ssp)

In [ ]:
sims = []

for step in range(500):
    new_ssp = ssp_map.voc["Zero"]
    clean_ssp = []
    for i in range(len(ground_ssp)):
        ground_ssp[i] *= T[i]
        ground_ssp[i].name = ""  # https://github.com/nengo/nengo-spa/pull/246
        clean_ssp.append(cleanup(ssp * ~ssp_map.voc[names[i]]))
        new_ssp += ssp_map.voc[names[i]] * clean_ssp[i] * T[i]
    ssp = new_ssp
        
    if step % 20 == 0:
        sim = np.sum(
            [ssp_map.compute_heatmap(clean_ssp[i])
             for i in range(len(clean_ssp))],
            axis=0)

        ground_sim = np.sum(
            [ssp_map.compute_heatmap(ground_ssp[i] * ~ssp_map.voc[names[i]])
             for i in range(len(ground_ssp))],
            axis=0)

        sims.append([sim, ground_sim, sim - ground_sim])

ani = heatmap_animation(list(zip(*sims)), figsize=(12, 4))
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))